In [1]:
import numpy as np
import pandas as pd
from utils.metrics import UnbiasedExponentialMovingAverage as uema

In [2]:
end_days = {
    "E8/P1/Bernoulli2/z2": 12,
    "E8/P1/Bernoulli3/z3": 12,
    "E8/P1/Bernoulli8/z8": 13,
    "E8/P1/Bernoulli9/z9": 14,
    "E7/P3/ESARSA3/z3": 14,
    "E7/P4/LinearESARSA6/z6": 13,
    "E7/P4/LinearESARSA8/z8": 13,
    "E7/P4/LinearESARSA9/z9": 13,
}
plants_per_zone = 36
steps_per_day = int(12 * 60 / 10 + 1)

In [3]:
def get_action_trace_history(action):
    # label traces by beta = 1 - alpha
    trace1 = uema(alpha=0.9)
    trace3 = uema(alpha=0.7)
    trace5 = uema(alpha=0.5)
    trace7 = uema(alpha=0.3)
    trace9 = uema(alpha=0.1)

    action_trace_history = []
    for a in action:
        trace1.update(a)
        trace3.update(a)
        trace5.update(a)
        trace7.update(a)
        trace9.update(a)

        action_trace_history.append(
            [
                a,
                trace1.compute().item(),
                trace3.compute().item(),
                trace5.compute().item(),
                trace7.compute().item(),
                trace9.compute().item(),
            ]
        )
    return action_trace_history

In [4]:
MEAN_DATA = []
for zone in end_days.keys():
    data_path = f"/home/lolanff/plant-rl/data/online/{zone}/raw.csv"
    df = pd.read_csv(data_path)

    actions = df["action.0"].to_numpy()
    actions = np.reshape(actions, (-1, plants_per_zone))
    action = np.array([np.mean(actions[i, :]) for i in range(actions.shape[0])])
    action = action / 0.398  # normalize action by the action.0 value for "bright"
    action_trace_history = get_action_trace_history(action)

    areas = df["mean_clean_area"].to_numpy()
    areas = np.reshape(areas, (-1, plants_per_zone))
    mean_area = np.array([np.mean(areas[i, :]) for i in range(areas.shape[0])])
    mean_area = mean_area / mean_area[0]  # normalize mean_area by initial value

    for i in range(end_days[zone] * steps_per_day):
        MEAN_DATA.append(
            [i, mean_area[i]] + action_trace_history[i] + [mean_area[i + 1]]
        )

In [5]:
np.save("./GP_data/mean_size_10min.npy", MEAN_DATA)